#### Point Map Visualization

Point map visualization is natively supported, if your table outputs longitude-latitude pairs. It is recommended to limit the query to not too many rows for performance reasons.

> [!NOTE]
> We are using Spatial Functions below, and as of 2025-07-16, you need to use Databricks Runtime 17.1+ for this. It is expected that this feature becomes available on Serverless Compute and DBSQL as well soon, see [here](https://www.linkedin.com/feed/update/urn:li:activity:7351276934338641920/).

In [0]:
%pip install geoarrow-pyarrow lonboard shapely --quiet

In [0]:
from lonboard import viz
import geoarrow.pyarrow as ga
import pyarrow as pa
import pyproj

In [0]:
spark.read.parquet("s3://overturemaps-us-west-2/release/2025-06-25.0/theme=places/type=place").createOrReplaceTempView("place")

In [0]:
%sql
with t as (
  select
    * except (geometry),
    st_geomfromwkb(geometry, 4326) native_geometry
  from
    place
)
select
  *,
  st_x(native_geometry) lon,
  st_y(native_geometry) lat,
from
  t
where
  bbox.xmin between 4.72 and 5.08
  and bbox.ymin between 52.27 and 52.44
  and categories.primary = "restaurant"
limit 10000

Databricks visualization. Run in Databricks to view.

In [0]:
dfa = _sqldf.toArrow()

In [0]:
# Assuming that all SRID's within the table are the same, which they should be as the SRID is part of the column type.
crs = pyproj.CRS.from_epsg(dfa['geometry'][0]['srid']).to_json()

In [0]:

field_chunks = [chunk.field('wkb') for chunk in dfa["geometry"].chunks]
dfaa = dfa.append_column("geoarrow", ga.with_crs(ga.as_geoarrow(pa.chunked_array(field_chunks)), crs))

In [0]:
viz(dfaa.select(["geoarrow"])).as_html()